## Milvus使用範例

In [2]:
# from pymilvus import model, MilvusClient, DataType, FieldSchema, CollectionSchema
# import pandas as pd
from pymilvus import model, MilvusClient, DataType, FieldSchema, CollectionSchema, connections, utility
from pymilvus.client.types import LoadState
import pandas as pd
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction

/Users/xrickliao/miniconda3/miniconda3/envs/llmenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# def init_embedding():
#     # 初始化嵌入模型
#     ef = model.DefaultEmbeddingFunction()  # 正確引用model內的方法
#     client = MilvusClient("./milvus_data.db")
#     return ef, client
'''
problemtype
module
severity
description
causeAnalysis
improve
experience
'''

'\nproblemtype\nmodule\nseverity\ndescription\ncauseAnalysis\nimprove\nexperience\n'

In [10]:
_dbName="dqe_kb_db"
_collectionName='qualityQA'
_collectionDesc="quailty_qa_collection"
_csv_src = "../../../source_data/DQE_Issues/DQE_Issue_total_corrected_v1.csv"
# jina_embeddings="/home/mapleleaf/LCJRepos/Embedding_Models/jina-embeddings-v2-base-zh/"
jina_embeddings = "/Users/xrickliao/WorkSpaces/Embedding_Models/jina-embeddings-v2-base-zh/"
alias = "default" # 使用的連線別名

In [11]:
'''
to-do:
    1.Adding whether dbName contain '.db' extension
    2.Checking whether collectionName has exist.
'''
    # 初始化组件
# ef = model.DefaultEmbeddingFunction()  # 确保已安装 pymilvus[model]
# openai_ef = model.dense.OpenAIEmbeddingFunction(
#     model_name='text-embedding-3-large', # Specify the model name
#     dimensions=512 # Set the embedding dimensionality according to MRL feature.
# )
ef = SentenceTransformerEmbeddingFunction(jina_embeddings, trust_remote_code=True)

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus",
    
) 



In [12]:
print(f"original dim of ef:{ef.dim}")

original dim of ef:768


In [13]:
current_collections = client.list_collections()
print(current_collections)
load_state = client.get_load_state(collection_name=_collectionName)#client.get_collection_stats(collection_name=_collectionName)
print(load_state)
print(LoadState.Loaded)
if load_state['state'] == LoadState.Loaded:
    print("collection has loaded")
# alias_to_use = getattr(client, 'alias') # 嘗試獲取內部 alias，否則用 'default'
# print(alias_to_use)
# load_state = utility.load_state(collection_name=_collectionName, using=alias_to_use)

[]
{'state': <LoadState: NotExist>}
Loaded


In [6]:
print(f"列出現在資料庫：{client.list_databases()}")
print("使用dqe_kb_db")
client.use_database(_dbName)
print(f"collection:{_collectionName}是否已建立:{client.has_collection(_collectionName)}")

列出現在資料庫：['default', 'dqe_kb_db']
使用dqe_kb_db
collection:qualityQA是否已建立:True


### Basic Operation: Database Creation, Collection Creation

#### database creation

In [16]:
# client.create_database(
#     db_name="dqe_kb_db"
# )

In [17]:
# check the encoding of csv file
import chardet

with open(_csv_src, 'rb') as f:
    result = chardet.detect(f.read(10000))  # 讀取前 10000 bytes
print(result['encoding'])

utf-8


In [18]:
# def create_dqe_db(dbName:str="dqe_milvus_data.db", collectionName:str="quality_issues", fields:list=None):
def create_dqe_collection(dbName:str=None, collectionName:str=None, collectionDesc:str=None, fieldList:list=None):
    # 定义字段结构（关键修正点）
    client.use_database(dbName)
    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="problemtype", dtype=DataType.VARCHAR, max_length=50),
        FieldSchema(name="module", dtype=DataType.VARCHAR, max_length=50),
        FieldSchema(name="severity", dtype=DataType.VARCHAR, max_length=1),
        FieldSchema(
                    name="description",
                    dtype=DataType.VARCHAR, 
                    max_length=2000,
                    enable_analyzer=True, # Whether to enable text analysis for this field
                    enable_match=True # Whether to enable text match
                   ),
        FieldSchema(name="causeAnalysis", dtype=DataType.VARCHAR, max_length=2000),
        FieldSchema(name="improve", dtype=DataType.VARCHAR, max_length=2000),
        FieldSchema(name="experience", dtype=DataType.VARCHAR, max_length=2000),
        # FieldSchema(name="judge", dtype=DataType.VARCHAR, max_length=2000),
        # FieldSchema(name="score", dtype=DataType.INT16),
        FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=ef.dim)
    ]
    
    # 创建集合（新API规范）
    # collection_name = "quality_issues"
    if client.has_collection(collectionName):
        client.drop_collection(collectionName)
    
    client.create_collection(
        collection_name=collectionName,
        schema=CollectionSchema(fields, description=collectionDesc),  # 单一路径传递schema
        # 不再需要单独传递fields参数
    )
    print(f"collection created:\n DB:{dbName}\ncollection name:{collectionName}");


In [19]:
create_dqe_collection(dbName=_dbName, collectionName=_collectionName, collectionDesc=_collectionDesc)
# client.use_database(db_name=_dbName)

collection created:
 DB:dqe_kb_db
collection name:qualityQA


#### delete collection

In [47]:
# def drop_collection(dbName:str=None, collectionName:str=None):
#     # client.use_database(db_name=dbName)
#     print(collectionName)
#     client.list_collections()
#     client.drop_collection(
#         collection_name=collectionName
#     )

In [10]:
# drop_collection(dbName=_dbName, collectionName=_collectionName)

In [16]:
# def create_index(dbName:str=None, collectionName:str=None, paramDict:dict=None):
#     # 索引創建參數
#     # client.use_database(db_name=dbName)
#     # client.load_collection(collection_name=collectionName)
#     index_params = client.prepare_index_params()
#     index_params.add_index(
#         field_name="vector", # use paramDict["field_name"]
#         index_type="IVF_FLAT", # use paramDict["index_type"]
#         metric_type="COSINE", # use paramDict["metric_type"]
#         params={"nlist": 256}  # 典型值 128-4096 #use paramDict["params"]
#     )
#     # 搜索參數對應關係
#     search_params = {
#         "nprobe": 32  # 值範圍 [1, nlist] #use paramDict["search_params"]
#     }

In [12]:
# create_index(dbName=_dbName, collectionName=_collectionName)

In [17]:
# client.list_collections()

#### 建立資料

In [20]:
def insert_items(csv_file:str=None, dbName:str=None, collectionName:str=None):
    # 数据插入示例
    client.use_database(db_name=dbName)
    df = pd.read_csv(csv_file,encoding='utf-8')
    # print(df.head())
    fill_df = df.fillna(0)
    data = [{
        "problemtype":row["问题来源"],
        "module": row["模块"],
        "severity": str(row["严重度"]),
        "description": row["问题现象描述"],
        "causeAnalysis": str(row["原因分析"]),
        "improve": row["改善对策"],
        "experience": str(row["经验萃取"]),
        # "judge": row["评审后优化"],
        # "score": int(row["评分"]),
        # "vector": ef.encode_documents([row["问题来源"],row["模块"],row["问题现象描述"]])[0]
        "vector": ef.encode_documents([row["问题现象描述"]])[0]
    } for _, row in fill_df.iterrows()]
    # print(data)
    client.insert(collectionName, data)
    
    print(f"成功插入 {len(df)} 条数据，向量维度={ef.dim}")

In [21]:
insert_items(csv_file=_csv_src, dbName=_dbName, collectionName=_collectionName)
##成功插入 108 条数据，向量维度=768

成功插入 108 条数据，向量维度=768


#### 建立index

In [22]:
def create_index(dbName:str=None, collectionName:str=None):
    print(f"db name:{dbName}\ncollection name:{collectionName}")
    client.use_database(db_name=dbName)
    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name="vector",
        index_type="IVF_FLAT",  # 改用支持的类型
        metric_type="COSINE",
        params={"nlist": 128}  # 典型参数配置
    )
    
    # 创建索引
    client.create_index(
        collection_name=collectionName,
        index_params=index_params
    )

In [23]:
# create_index(dbName=_dbName, collectionName=_collectionName)

db name:dqe_kb_db
collection name:qualityQA


## Test Codes

#### loading db and collection

In [25]:
# df = pd.read_csv(_csv_src)
import duckdb, polars as pl

In [26]:
csvdb = "../duckdb/db/csvdb.duckdb"
con = duckdb.connect(csvdb)
pl_df = con.execute("SELECT * FROM my_table").pl()
con.close()

In [27]:
print(pl_df)

shape: (108, 8)
┌──────────┬──────┬────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│ 问题来源 ┆ 模块 ┆ 严重度 ┆ 问题现象描述 ┆ 原因分析     ┆ 改善对策    ┆ 经验萃取    ┆ column7     │
│ ---      ┆ ---  ┆ ---    ┆ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         │
│ str      ┆ str  ┆ str    ┆ str          ┆ str          ┆ str         ┆ str         ┆ str         │
╞══════════╪══════╪════════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
│ PVT      ┆ 结构 ┆ B      ┆ 摄像头偏心   ┆ 1.大公模项目 ┆ 1.设计时需  ┆ 1.从项目开  ┆ null        │
│          ┆      ┆        ┆              ┆ 设计公差累计 ┆ 要完全定位  ┆ 始立项与结  ┆             │
│          ┆      ┆        ┆              ┆ 为0.8mm      ┆ 摄像头,不使 ┆ 构设计需提  ┆             │
│          ┆      ┆        ┆              ┆ 2.与客户标准 ┆ 用治具/人为 ┆ 前定义偏心  ┆             │
│          ┆      ┆        ┆              ┆ 未达成…      ┆ 等矫正措施  ┆ 标准        ┆             │
│          ┆      ┆        ┆              ┆              ┆ 2…          ┆ 

In [16]:
def test_basic_operations():
    # 连接验证
    client.use_database(db_name=_dbName)
    client.load_collection(_collectionName)
    assert client.has_collection(_collectionName), "集合创建失败"
    print(client.has_collection(_collectionName))
    
    # 数据量验证
    count = client.query(_collectionName, filter="", output_fields=["count(*)"])[0]["count(*)"]
    assert count == len(df), f"数据量不符 ({count} vs {len(df)})"
    
    # 向量维度验证
    collection_info = client.describe_collection(_collectionName)
    
    # assert collection_info["vector_field"]["dim"] == ef.dim, "向量维度错误"
    vector_fields = [f for f in collection_info['fields'] if f['type'] == DataType.FLOAT_VECTOR]
    assert len(vector_fields) > 0, "未找到向量字段"
    assert vector_fields[0]['params']['dim'] == ef.dim, "向量維度錯誤"
    
    # 随机抽样验证
    sample = client.query(_collectionName, filter="pk < 5", output_fields=["*"])
    for item in sample:
        assert len(item["vector"]) == ef.dim, "向量长度异常"
        assert item["description"], "描述字段为空"

# test_basic_operations()


#### Test different searches

In [12]:
def test_hybrid_search_1():
    client.use_database(db_name=_dbName)
    exists = client.has_collection(_collectionName)
    print(f"if the collection:{_collectionName} is existed:{exists}")
    if exists:
        _load_state = client.get_load_state(collection_name=_collectionName)
        print(f"if the collection:{_collectionName} loaded:{_load_state}")
        if _load_state['state'] != LoadState.Loaded:
            client.load_collection(_collectionName)
    index_info = client.describe_index(_collectionName, "vector")
    # print(index_info)
    assert index_info["index_type"] == "IVF_FLAT"
    # assert index_info["nlist"] == 256
    assert int(index_info["nlist"]) == 128
    
    # 執行混合查詢
    # query = "电池鼓包问题"
    query = "比亚迪"
    encoded_query = ef.encode_documents([query])
    # print(encoded_query)
    query_vec = encoded_query[0]
    results = client.search(
        collection_name=_collectionName,
        data=[query_vec],
        output_fields=['description'],
        # filter="severity == 'B'",
        # limit=3,
        search_params= {"metric_type": "COSINE"}#{"nprobe":32}
    )
    # 精度驗證
    print(f"results:searched rows:{len(results[0])}\n{results}")
    distances = [hit['distance'] for hit in results[0]]
    # assert max(distances) > 0.7  # 根據實際數據調整閾值

test_hybrid_search_1()

if the collection:qualityQA is existed:True


ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>

In [25]:
# import time

# def test_performance():
#     # 查询延迟测试
#     start = time.time()
#     client.query(collection_name, filter="pk < 100", limit=100)
#     print(f"标量查询延迟: {time.time()-start:.4f}s")

#     # 向量搜索压力测试
#     test_vectors = [ef.random_vector(ef.dim) for _ in range(100)]
#     start = time.time()
#     client.search(collection_name, data=test_vectors, limit=3)
#     print(f"批量向量搜索延迟: {time.time()-start:.4f}s")

# test_performance()